# Environment

In [353]:
import matplotlib
matplotlib.use('TkAgg')

In [354]:
import torch
import copy
import numpy as np
import numpy.random as npr
import random


In [ ]:
# Define the grid world environment
class GridWorldEnvironment:
    def __init__(self, size=5, agents_num=4):
        self.size = size
        self.agents_num = agents_num
        self.agents_positions = {}  # agent position
        self.agents_reached_A = {}  # if agents get item
        self.A_position = None
        self.B_position = (size - 1, size - 1)  # fixed location of B
        self.directions = ["up", "down", "left", "right"]
        self.total_collisions = 0
        self.total_steps = 0
        self.agents_idx = list(range(agents_num))
        self._reset()

    def _reset(self):
        """
        Reset the environment to its initial state.
        """
        # initialize A position
        self.A_position = (
            npr.randint(0, self.size - 1),
            npr.randint(0, self.size - 1),
        )
        # ensure A and B are not in the same position
        while self.A_position == self.B_position:
            self.A_position = (
                npr.randint(0, self.size - 1),
                npr.randint(0, self.size - 1),
            )

        # initialize agents' positions and reached_A status
        self.agents_positions = {}
        self.agents_reached_A = {}
        for idx in self.agents_idx:
            if npr.rand() > 0.5:
                self.agents_positions[idx] = self.A_position
                self.agents_reached_A[idx] = True
            else:
                self.agents_positions[idx] = self.B_position
                self.agents_reached_A[idx] = False

        self.total_collisions = 0
        self.total_steps = 0

    def _get_destination(self, agent_idx):
        """
        Get the destination position(A or B)
        """
        return "B" if self.agents_reached_A[agent_idx] else "A"

    def _find_nearby_collision_agents(self, agent_id):
        """
        Find nearby agents that might collide.
        """
        y, x = self.agents_positions[agent_id]
        destination_cur = self._get_destination(agent_id)
        nearby_agents = [
            (-1, -1),
            (-1, 0),
            (-1, 1),
            (0, -1),
            (0, 1),
            (1, -1),
            (1, 0),
            (1, 1),
        ]
        collision_status = []
        for dy, dx in nearby_agents:
            new_y, new_x = y + dy, x + dx
            # Check if new position is valid
            if 0 <= new_y < self.size and 0 <= new_x < self.size:
                has_agent = 0
                for other_agent_id in self.agents_idx:
                    if (
                        other_agent_id != agent_id
                        and self.agents_positions[other_agent_id] == (new_y, new_x)
                        and self._get_destination(other_agent_id) != destination_cur
                    ):  # agents are going to the same destination would cause collision
                        has_agent = 1
                collision_status.append(has_agent)
            else:
                collision_status.append(0)
        return collision_status

    def get_state(self, agent_idx):
        """
        Get the state of the environment for a specific agent.
        """
        position = self.agents_positions[agent_idx]
        reached_A = self.agents_reached_A[agent_idx]
        # manhattan_distance_to_A = (
        #     (self.A_position[0] - position[0]),
        #     (self.A_position[1] - position[1]),
        # )
        collision_agents = self._find_nearby_collision_agents(agent_idx)

        return np.array(
            [
                position[0],
                position[1],
                self.A_position[0],
                self.A_position[1],
                # manhattan_distance_to_A[0],
                # manhattan_distance_to_A[1],
                reached_A,
                *collision_agents,
            ]
        )

    def _check_done(self, agent_idx):
        """
        Check if the agent has reached its destination.
        """
        return (
            self.agents_positions[agent_idx] == self.B_position
            and self.agents_reached_A[agent_idx]
        )

    def take_action(self, action_dict):
        """
        Take an action in the environment and return the next state, reward and collosions.
        """
        planned_actions = {}  # {action_idx: action}
        wall_collisions = []  # number of hitting wall

        for agent_idx, action in action_dict.items():
            y, x = self.agents_positions[agent_idx]
            if self.directions[action] == "up":
                new_y, new_x = y - 1, x
            elif self.directions[action] == "down":
                new_y, new_x = y + 1, x
            elif self.directions[action] == "left":
                new_y, new_x = y, x - 1
            elif self.directions[action] == "right":
                new_y, new_x = y, x + 1

            # check valid
            if 0 <= new_y < self.size and 0 <= new_x < self.size:
                planned_actions[agent_idx] = (new_y, new_x)  # move
                wall_collisions.append(False)
            else:
                planned_actions[agent_idx] = (y, x)  # not move
                wall_collisions.append(True)

        # check collision
        next_positions = copy.deepcopy(self.agents_positions)
        for idx in self.agents_idx:
            next_positions[idx] = planned_actions[idx]

        collisions = 0  # number of head-on collisions
        position_to_agents = {}
        for agent_idx, pos in next_positions.items():
            if pos not in position_to_agents:
                position_to_agents[pos] = []
            position_to_agents[pos].append(agent_idx)
        agents_collisions = set()
        for pos, agents_cur in position_to_agents.items():
            if pos == self.A_position or pos == self.B_position:
                continue  # ignore overlap at A or B
            if len(agents_cur) > 1:
                dirs = [self._get_destination(a) for a in agents_cur]
                if "B" in dirs and "A" in dirs:
                    collisions += 1
                    agents_collisions.update(agents_cur)

        # check swamp collision
        for i in self.agents_idx:
            for j in self.agents_idx:
                if i < j:  # 避免重复检查
                    current_pos_i = self.agents_positions[i]
                    next_pos_i = planned_actions[i]
                    current_pos_j = self.agents_positions[j]
                    next_pos_j = planned_actions[j]
                    # 判断是否为对向穿越
                    if (
                        next_pos_i == current_pos_j
                        and next_pos_j == current_pos_i
                        and self._get_destination(i) != self._get_destination(j)
                        and current_pos_i != self.A_position
                        and current_pos_i != self.B_position
                        and current_pos_j != self.A_position
                        and current_pos_j != self.B_position
                    ):
                        collisions += 1
                        agents_collisions.add(i)
                        agents_collisions.add(j)

        # calculate rewards
        rewards = {}
        for agent_idx in self.agents_idx:
            reward = 0
            # ===== 我的reward逻辑 =====
            # if wall_collisions[agent_idx]:
            #     reward += -10  # hitting wall penalty
            # # else:
            # #     reward += -1  # step cost

            # if agent_idx in agents_collisions:
            #     reward += -200  # Collision penalty
            # else:
            #     reward += 2  # no collision reward

            # location = self.agents_positions[agent_idx]
            next_location = next_positions[agent_idx]
            # # distance reward to A or B
            # goal = (
            #     self.A_position
            #     if not self.agents_reached_A[agent_idx]
            #     else self.B_position
            # )
            # prev_dist = abs(location[0] - goal[0]) + abs(location[1] - goal[1])
            # curr_dist = abs(next_location[0] - goal[0]) + abs(
            #     next_location[1] - goal[1]
            # )
            # reward += (
            #     max(0, (prev_dist - curr_dist)) * 5
            # )  # every step close to goal, acculate reward by +5

            # if self.agents_reached_A[agent_idx]:
            #     # 如果之前已经取货 现在到B 就算完成一次交付
            #     if next_location == self.B_position:
            #         reward += 300  # delivery success
            #         # self.agents_reached_A[agent_idx] = False
            # else:
            #     # 如果还没取货 现在到A 就给取货奖励
            #     if next_location == self.A_position:
            #         reward += 100  # pickup success
            #         # self.agents_reached_A[agent_idx] = True
            # ===== 我的reward逻辑 =====
            

            # 严格合规的reward逻辑
            if self.agents_reached_A[agent_idx] == False:
                if next_location == self.A_position:
                    self.reached_A = True
                    reward += 100
                else:
                    reward = -1
            elif self.agents_reached_A[agent_idx] and next_location == self.B_position:
                reward += 300
            else:
                reward = -1

            rewards[agent_idx] = reward  # store reward

        # update agents' positions
        self.agents_positions = next_positions

        # accumulate total collisions and steps
        self.total_collisions += collisions
        self.total_steps += self.agents_num

        # Update item-carrying status after moving
        for agent_idx in self.agents_idx:
            if (
                self.agents_reached_A[agent_idx]
                and self.agents_positions[agent_idx] == self.B_position
            ):
                self.agents_reached_A[agent_idx] = False  # delivered item at B
            elif (not self.agents_reached_A[agent_idx]) and self.agents_positions[
                agent_idx
            ] == self.A_position:
                self.agents_reached_A[agent_idx] = True  # picked up item at A

        # format next state
        next_states = {}
        for agent_idx in self.agents_idx:
            next_states[agent_idx] = self.get_state(agent_idx)

        # LOG
        # print(f"Agent {agent_idx} | Position: {next_location} | Reward: {reward}")

        return next_states, rewards, collisions

# Agent

In [356]:
# deep q-learning agent
class Agent:
    def __init__(
        self,
        statespace_size,
        action_size,
        gamma=0.99,
        epsilon=1.0,
        epsilon_decay=0.995,
        min_epsilon=0.1,
        batch_size=256,
        replay_buffer_size=50000,
        lr=0.001,
        copy_frequency=100,
    ):
        self.statespace_size = statespace_size
        self.action_size = action_size
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon
        self.batch_size = batch_size
        self.replay_buffer_size = replay_buffer_size
        self.lr = lr
        self.copy_frequency = copy_frequency

        self.steps = 0  # count agent's steps
        self.replay_buffer = []  # memory
        self.replay_buffer_size = replay_buffer_size  # memory size

        # initialize the DQN
        self.model, self.model2, self.optimizer, self.loss_fn = self.prepare_torch()

    def prepare_torch(self):
        l1, l2, l3, l4 = self.statespace_size, 128, 128, self.action_size

        model = torch.nn.Sequential(
            torch.nn.Linear(l1, l2),
            torch.nn.ReLU(),
            torch.nn.Linear(l2, l3),
            torch.nn.ReLU(),
            torch.nn.Linear(l3, l4),
        )
        model2 = copy.deepcopy(model)
        model2.load_state_dict(model.state_dict())
        loss_fn = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=self.lr)
        return model, model2, optimizer, loss_fn

    def update_target(self):
        self.model2.load_state_dict(self.model.state_dict())

    def get_qvals(self, state):
        state_tensor = torch.from_numpy(state).float()
        qvals_torch = self.model(state_tensor)
        qvals = qvals_torch.detach().numpy()
        return qvals

    def get_maxQ(self, s):
        return torch.max(self.model2(torch.from_numpy(s).float())).detach().numpy()

    def get_action(self, state):
        if npr.uniform() < self.epsilon:
            action = npr.choice(self.action_size)
        else:
            qvals = self.get_qvals(state)
            action = np.argmax(qvals)
        return action

    def store_transition(self, state, action, reward, next_state):
        """
        Store the transition in the replay buffer.
        """
        if len(self.replay_buffer) >= self.replay_buffer_size:
            # random remove sample
            remove_idx = npr.randint(0, len(self.replay_buffer))
            self.replay_buffer.pop(remove_idx)
        self.replay_buffer.append((state, action, reward, next_state))

    def train_one_step(self, states, actions, targets):
        # convert the states and actions and targets to tensors
        states = np.array(states)
        actions = np.array(actions)
        targets = np.array(targets)

        state_batch = torch.tensor(states, dtype=torch.float32)
        action_batch = torch.tensor(actions, dtype=torch.long)
        target_batch = torch.tensor(targets, dtype=torch.float32)

        # get Q-values for the current states
        q_values = self.model(state_batch)
        predicted_q_values = q_values.gather(1, action_batch.unsqueeze(1)).squeeze()

        # calculate the loss
        loss = self.loss_fn(predicted_q_values, target_batch)

        # backpropagation
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()

    def train(self):
        """
        Train the agent using the replay buffer.
        """
        if len(self.replay_buffer) < self.batch_size:
            return  # samples not enough

        # sample a batch from the replay buffer
        minibatch = random.sample(
            self.replay_buffer,
            self.batch_size,
        )
        states, actions, rewards, next_states = zip(*minibatch)

        # TD targets
        targets = []
        for i in range(len(minibatch)):
            next_maxQ = self.get_maxQ(next_states[i])
            action_target = rewards[i] + self.gamma * next_maxQ
            targets.append(action_target)

        # train the model
        loss = self.train_one_step(states, actions, targets)

        # update network periodically
        self.steps += 1
        if self.steps % self.copy_frequency == 0:
            self.update_target()

        return loss

    # decay epsilon
    def decay_epsilon(self):
        self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)

# Training

In [357]:
import time


def train_agents(
    agent, env, max_steps=1500000, max_collisions=4000, max_walltime=600, verbose=True
):
    print("train_agents triggered")
    """Training each agent in the environment."""
    # start time
    start_time = time.time()

    # global variables
    total_collisions = 0
    total_steps = 0
    episode = 0

    while total_collisions <= max_collisions and total_steps <= max_steps:
        if time.time() - start_time > max_walltime:
            print("===== Time limit exceeded. =====")
            break

        # episode max steps
        steps_this_episode = 0

        # initialize the environment
        env._reset()

        # initial states
        states = {agent_idx: env.get_state(agent_idx) for agent_idx in env.agents_idx}

        # episode finish flag
        done = False

        TRAIN_INTERVAL = 5  # 每 5 个环境步训练一次
        step_since_train = 0

        while not done:
            steps_this_episode += len(env.agents_idx)
            # force end if episode max steps exceeded. (0,0)->(4,4)
            # max 25 steps, try set maximum 75 steps
            if steps_this_episode >= 75 * len(env.agents_idx):
                done = True
                break

            actions_dict = {}
            for agent_idx in sorted(env.agents_idx):  # central clock - fix order
                action = agent.get_action(states[agent_idx])
                actions_dict[agent_idx] = action

            # take action in the environment
            next_states, rewards, collisions = env.take_action(actions_dict)
            # print(f"[Step {total_steps + len(env.agents_idx)}] rewards: {rewards}")

            # store transition in replay buffer
            for agent_idx in sorted(env.agents_idx):
                state = states[agent_idx]
                action = actions_dict[agent_idx]
                reward = rewards[agent_idx]
                next_state = next_states[agent_idx]
                agent.store_transition(state, action, reward, next_state)

            step_since_train += 1
            # train the agent
            if total_steps < 5000: # 前5000步多训练，练习避免碰撞
                agent.train()
            elif (
                len(agent.replay_buffer) >= agent.batch_size
                and step_since_train >= TRAIN_INTERVAL
            ):
                agent.train()
                step_since_train = 0

            total_collisions += collisions
            total_steps += len(env.agents_idx)

            # add step-based epsilon decay
            if total_steps % 1000 == 0:
                agent.epsilon = max(agent.min_epsilon, agent.epsilon * 0.997)

            # log
            if verbose and total_steps % 5000 == 0:
                elapsed = time.time() - start_time
                print(
                    f"Steps: {total_steps}/{max_steps}, "
                    f"Collisions: {total_collisions}/{max_collisions}, "
                    f"Epsilon: {agent.epsilon:.3f}, "
                    f"Time Elapsed: {elapsed:.1f}s"
                )

            # check if any agent has done the task
            for i in env.agents_idx:
                if env._check_done(i):
                    done = True
                    break

            # check if the training should stop
            if (
                total_steps >= max_steps
                or total_collisions >= max_collisions
                or time.time() - start_time > max_walltime
            ):
                done = True
                break

            # update the states
            states = next_states

        # one episode end
        episode += 1
        if episode % 10 == 0:  # update epsilon every 10 episodes
            agent.decay_epsilon()

        # log
        if verbose and episode % 10 == 0:
            elapsed = time.time() - start_time
            print(
                f"Episode: {episode}, "
                f"TotalSteps: {total_steps}, "
                f"TotalCollisions: {total_collisions}, "
                f"Epsilon: {agent.epsilon:.3f}, "
                f"Elapsed: {elapsed:.1f}s"
            )

        # if verbose and total_steps % 5000 == 0:
        #     elapsed_time = time.time() - start_time
        #     print(
        #         f"Steps: {total_steps}/{max_steps}, Collisions: {total_collisions}/{max_collisions}, Epsilon: {agent.epsilon:.3f}, Time Elapsed: {elapsed_time:.2f}s"
        #     )

    print("Training completed.")
    print(f"Total steps: {total_steps}")
    print(f"Total collisions: {total_collisions}")
    print(f"Final epsilon: {agent.epsilon:.3f}")

# Test

In [358]:
def test_agents(agent, env, test_times=100, max_steps=25, step_verbose=True):
    """
    Test the trained agent in the environment.
    """
    # initialize the parmeters
    original_epsilon = agent.epsilon
    agent.epsilon = 0
    success_times = 0
    total_steps_successful = 0
    total_collisions = 0

    for run in range(test_times):
        # reset the environment
        env._reset()

        # all agents start at B(as the assingment requirenment says)
        for agent_idx in env.agents_idx:
            env.agents_positions[agent_idx] = env.B_position
            env.agents_reached_A[agent_idx] = False

        # initial states
        states = {agent_idx: env.get_state(agent_idx) for agent_idx in env.agents_idx}

        # initialize the variables
        delivery_success = {agent_idx: False for agent_idx in env.agents_idx}
        steps_taken = {agent_idx: 0 for agent_idx in env.agents_idx}
        collisions_happened = False

        for step in range(max_steps):
            actions_dict = {}
            for agent_idx in sorted(env.agents_idx): # central clock - fix order
                action = agent.get_action(states[agent_idx])
                actions_dict[agent_idx] = action

            # take action in the environment
            next_states, rewards, collisions = env.take_action(actions_dict)
            # update the states
            states = next_states
            # update stepts taken
            for agent_idx in env.agents_idx:
                steps_taken[agent_idx] += 1

            # check collisions
            if collisions > 0:
                collisions_happened = True

            # check delivery success
            for agent_idx in env.agents_idx:
                position = env.agents_positions[agent_idx]
                reached_A = env.agents_reached_A[agent_idx]
                if position == env.B_position and not reached_A:
                    delivery_success[agent_idx] = True

            # check if all agents have successfully delivered
            if all(delivery_success.values()):
                break


        # record successful deliveries
        for agent_idx in sorted(env.agents_idx):
            if (
                delivery_success[agent_idx]
                and not collisions_happened
                and steps_taken[agent_idx] <= max_steps
            ):
                success_times += 1
                total_steps_successful += steps_taken[agent_idx]

        # record collisions
        if collisions_happened:
            total_collisions += 1

        # print step verbose
        if step_verbose:
            print(
                f"[TEST] Run {run + 1}/{test_times}: Success deliveries = {sum(delivery_success.values())}, Collisions: {collisions_happened}"
            )

    # restore the epsilon
    agent.epsilon = original_epsilon

    # test indicators
    total_possible_deliveries = len(env.agents_idx) * test_times
    success_rate = (success_times / total_possible_deliveries) * 100
    avg_steps = total_steps_successful / success_times if success_times > 0 else 0
    collisions_rate = (total_collisions / test_times) * 100

    # test summary
    print("\n===== Test Summary =====")
    print(f"Success rate: {success_rate:.2f}%")
    print(f"Average steps per successful delivery: {avg_steps:.2f}")
    print(f"Total Collision: {total_collisions}")

    return {
        "success_rate": success_rate,
        "avg_steps": avg_steps,
        "total_collisions": total_collisions,
        "collisions_rate": collisions_rate,
    }

## Test process

In [359]:
import numpy as np
import matplotlib.pyplot as plt 
import time

# setup the environment
test_env = GridWorldEnvironment()
test_state = test_env.get_state(0)
statespace_size = test_state.shape[0]
print(f"State space size: {statespace_size}")
action_size = len(test_env.directions)

# trainning parameters
trainning_steps = [500000, 1000000, 1500000]
test_runs = 100
repeat_times = 2

# results metrics
results = {
    "trainning_steps": [],
    "success_rate_mean": [],
    "success_rate_std": [],
    "average_steps_mean": [],
    "average_steps_std": [],
    "collision_rate_mean": [],
    "collision_rate_std": [],
}

# Test 1: trainning steps & performance
for steps in trainning_steps:
    print("\n===== Training with steps: {} =====".format(steps))

    success_rates = []
    average_steps_list = []
    collision_rates = []

    # initialize the agent and environment
    env = GridWorldEnvironment()
    agent = Agent(
        statespace_size,
        action_size,
    )

    # train the agent
    train_agents(
        agent,
        env,
        max_steps=steps,
        verbose=False,
    )

    # test the agent
    metrics = test_agents(
        agent, env, test_times=test_runs, max_steps=25, step_verbose=False
    )

    # record the results
    success_rates.append(metrics["success_rate"])
    average_steps_list.append(metrics["avg_steps"])
    collision_rates.append(metrics["total_collisions"])
    total_collisions = metrics["total_collisions"]

# calculate and store the results
results["trainning_steps"].append(steps)
results["success_rate_mean"].append(np.mean(success_rates))
results["success_rate_std"].append(np.std(success_rates))
results["average_steps_mean"].append(np.mean(average_steps_list))
results["average_steps_std"].append(np.std(average_steps_list))
# results["total_collisions"].append(total_collisions)
results["collision_rate_mean"].append(np.mean(collision_rates))
results["collision_rate_std"].append(np.std(collision_rates))

# plot
plt.figure(figsize=(14, 8))
plt.suptitle("Test 1: Training Steps vs. Performance")

plt.subplot(2, 2, 1)
plt.errorbar(
    results["trainning_steps"],
    results["success_rate_mean"],
    yerr=results["success_rate_std"],
    fmt="o-",
    capsize=5,
)
plt.title("Success Rate")
plt.xlabel("Training Steps")
plt.ylabel("Success Rate (%)")

plt.subplot(2, 2, 2)
plt.errorbar(
    results["trainning_steps"],
    results["average_steps_mean"],
    yerr=results["average_steps_std"],
    fmt="o-",
    capsize=5,
)
plt.title("Average Steps")
plt.xlabel("Training Steps")
plt.ylabel("Average Steps")

plt.subplot(2, 2, 3)
plt.errorbar(
    results["trainning_steps"],
    results["collision_rate_mean"],
    yerr=results["collision_rate_std"],
    fmt="o-",
    capsize=5,
)
plt.title("Collision Rate")
plt.xlabel("Training Steps")
plt.ylabel("Collision Rate (%)")
plt.tight_layout()
plt.show()


# Test 2: Generalization
print("\n===== Generalization Test =====")
env = GridWorldEnvironment()
agent = Agent(
    statespace_size,
    action_size,
)
train_agents(
    agent,
    env,
    max_steps=1500000,
    max_collisions=4000,
    verbose=True,
)

metrics = test_agents(agent, env, test_times=200, max_steps=25, step_verbose=False)

print("\n===== Test 2 Final test metrics =====")
print(f"Success rate: {metrics['success_rate']:.2f}%")
print(f"Average steps: {metrics['avg_steps']:.2f}%")
print(f"Total collisions: {metrics['total_collisions']}")

State space size: 13

===== Training with steps: 500000 =====
train_agents triggered


NameError: name 'next_location' is not defined